# POC - 1 2 3 Custom MXNet Container – What does it mean? and autoscaling using 2 ways

In [ ]:
#Checkout the inference.py 

In [ ]:
!pygmentize inference.py

In [ ]:
!cat Dockerfile.gpu

In [ ]:
#Package artifact and wrap it up in a tar.gz

#!(cd model_untarred/ && tar -czvf /tmp/model-resnet-152-inference.tar.gz *)
#!aws s3 cp /tmp/model-resnet-152-inference.tar.gz s3://ar54/super_models/
#!aws s3 ls s3://ar54/super_models/
#!aws s3 ls s3://ar54/super_models/model-resnet-152-inference.tar.gz

In [ ]:
!cat build_and_push.sh

In [ ]:
!./build_and_push.sh 

In [ ]:
import sagemaker
from sagemaker.mxnet import MXNetModel

role = sagemaker.session.get_execution_role()

model_data="s3://ar54/super_models/model-resnet-152-inference.tar.gz"

model = MXNetModel(
    model_data=model_data,
    role=role,
    image="308412838853.dkr.ecr.us-east-2.amazonaws.com/chazarey-mxnet-serving-160-gpu-py3:latest",
    entry_point="inference.py",
    py_version='py3',
    framework_version='1.6.0',
    enable_cloudwatch_metrics=True
)
# ml.g4dn.16xlarge ml.p3.16xlarge ml.p2.xlarge
#predictor = model.deploy(instance_type="local_gpu", initial_instance_count=1)
predictor = model.deploy(instance_type='ml.g4dn.xlarge', initial_instance_count=1)

In [ ]:
print(predictor.endpoint)
#print avg of 100 runs below usig numpy

In [ ]:
from matplotlib.pyplot import imshow
import os
import io
import subprocess
import PIL
from PIL import Image
import requests
import random
import io
import numpy as np
import json
from sagemaker.predictor import StringDeserializer
import time
import image_pb2 as impb
import ast
from IPython.display import clear_output
from sagemaker.predictor import RealTimePredictor, json_deserializer

with open('synset.txt', 'r') as f:
  labels = [l.rstrip() for l in f]

impath = '/home/ec2-user/SageMaker/dataset/256_ObjectCategoriesimage'
files = os.listdir(impath)

while True:
    index = random.randrange(0, len(files))
    #print(files[index])
    file_name = impath + '/' + files[index]
    
    try:
        with open(file_name, 'rb') as f:
            payload = f.read()
            #payload = bytearray(payload)
    except:
        pass
        
    image_packet = impb.PBImage()
    image_packet.image_data = payload

    predictor.serializer = None
    predictor.deserializer = StringDeserializer()
    predictor.accept = None
    predictor.content_type = 'application/octet-stream'

    %time response = predictor.predict(image_packet.SerializeToString())            
    
    result = ast.literal_eval(response)
    index = np.argmax(result)
    #print(index)
    pil_im = Image.open(file_name, 'r')    
    imshow(np.asarray(pil_im))
    print('\n'+labels[index]+'\n')
    clear_output(wait=True)
    
